#### Documentation: 
- [scikit-learn](http://scikit-learn.org/stable/user_guide.html)
- [pandas](http://pandas.pydata.org/pandas-docs/stable/)
- [numpy](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.html)
- [matplotlib](https://matplotlib.org/2.0.2/users/pyplot_tutorial.html)
- [TPOT](https://www.datacamp.com/community/tutorials/tpot-machine-learning-python)
- [skopt](https://scikit-optimize.github.io/)

<font color = "#CC3D3D">
# Cross-Validation

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(digits.data[50].reshape(8,8), cmap=plt.cm.gray_r)
digits.target[50]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, random_state=0)

In [ ]:
X_train

<font color = "darkgreen">
### k-NN(Nearest Neighber) #####
</font>
<img align="left" src="https://t1.daumcdn.net/cfile/tistory/21231436593671A33D" width=500 height=400 alt="Decision Tree">

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

model = KNeighborsClassifier()
#model = LogisticRegression()
#model = DecisionTreeClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train); scores

In [ ]:
# Change k
scores = cross_val_score(model, X_train, y_train, cv=5); scores

In [ ]:
print("Mean: {:.3f}\nStd: {:.3f}\nMin: {:.3f}\nMax: {:.3f}".format(
    scores.mean(), scores.std(), scores.min(), scores.max()))

<font color = "darkgreen">
### LOOCV(Leave-One-Out Cross-Validation)
<img align="left" src="https://www.researchgate.net/profile/Nikolaos_Chlis/publication/266617511/figure/fig11/AS:295705362092036@1447513060277/Leave-One-Out-Cross-Validation.png" width=500 height=400 alt="Decision Tree">

In [ ]:
# LOOCV is very time-consuming => useful in small data

from sklearn.model_selection import LeaveOneOut

scores = cross_val_score(model, X_train, y_train, cv=LeaveOneOut())
scores.mean()

<font color = "darkgreen">
### Shuffle-Split Cross-Validation
<p>   
<img align="left" src="https://t1.daumcdn.net/cfile/tistory/99B45B3B5A5BE4D715" width=900 height=700 alt="Decision Tree">

In [ ]:
from sklearn.model_selection import ShuffleSplit

sscv = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10)
scores = cross_val_score(model, X_train, y_train, cv=sscv)
scores.mean()

<br><font color = "#CC3D3D">
## Model Tuning ##
<br>
<img align="left" src="https://i1.wp.com/hugrypiggykim.com/wp-content/uploads/2017/09/hyper-parameter-search.jpg?resize=698%2C242" width=800 height=600 alt="Decision Tree">

<font color = "darkgreen">
### Grid Search CV 

##### Set the parameters for grid search #####

In [ ]:
# param_grid: dictionary with parameters names as keys and
# lists of parameter settings to try as values

param_grid = {'n_neighbors': range(1,5),
              'weights': ['uniform','distance']}
param_grid

##### Grid search with cross-validation ####

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, n_jobs=-1)

In [ ]:
%%time
# grid search is very time-consuming

grid_search.fit(X_train, y_train)

##### Evaluate the model with best parameters ####

In [ ]:
grid_search.score(X_test, y_test), KNeighborsClassifier().fit(X_train, y_train).score(X_test, y_test)

In [ ]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best CV score: {:.2f}".format(grid_search.best_score_))

In [ ]:
print("Best estimator:\n{}".format(grid_search.best_estimator_))

##### When the parameters are asymmetric #####

In [ ]:
# In the case of SVM

param_grid = [{'kernel': ['rbf'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100],
               'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
              {'kernel': ['linear'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

<font color = "darkgreen">
### Random Search CV

##### Set the parameters for random search #####

In [ ]:
#from scipy.stats import uniform as sp_rand

from scipy.stats import randint as sp_randint

param_grid = {'n_neighbors': sp_randint(1, 100),  # sp_randint(1, 100).rvs(10)
              'weights': ['uniform','distance']}
param_grid

##### Random search with cross-validation ####

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rand_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_grid, n_iter=20, random_state=1)

In [ ]:
rand_search.fit(X_train, y_train)

##### Evaluate the model with best parameters ####

In [ ]:
rand_search.score(X_test, y_test)

In [ ]:
print("Best estimator:\n{}".format(rand_search.best_estimator_))

<font color = "darkgreen">
### AutoML: The Tree-Based Pipeline Optimization Tool (TPOT)  
<br>
<img align="left" src="https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1537396029/output_2_0_d7uh0v.png" width=800 height=600 alt="Decision Tree">

##### Installation

In [ ]:
!pip install tpot

##### Optimizes machine learning pipelines using genetic programming

In [ ]:
from tpot import TPOTClassifier
#from tpot import TPOTRegressor

tpot = TPOTClassifier(verbosity=2, max_time_mins=5, max_eval_time_mins=0.05, population_size=30, n_jobs=-1, random_state=1)
tpot.fit(X_train, y_train)

In [ ]:
tpot.score(X_test, y_test)

##### Export the corresponding Python code for the optimized pipeline

In [ ]:
tpot.export('tpot_MNIST_pipeline.py')

In [ ]:
# %load tpot_MNIST_pipeline.py

<font color = "darkgreen">
### Scikit-Optimize, or skopt

##### Installation

In [ ]:
# !pip install scikit-optimize

##### Bayesian optimization over hyper parameters
- In contrast to GridSearchCV, 
- not all parameter values are tried out, 
- but rather a fixed number of parameter settings is sampled from the specified distributions.

<img align="left" src="https://cdn-images-1.medium.com/max/1600/1*2qDZxQkRoP28CidZtoT-gQ.png" width=400 height=300 alt="Decision Tree">

- Q) If you were picking the next number of trees to evaluate, where would you concentrate?
- A) https://www.quora.com/How-does-Bayesian-optimization-work

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
space = {
    'n_neighbors': Integer(1, 100),
    'weights': Categorical(['uniform','distance']),
    # Real(1e-6, 1e+6, prior='log-uniform')
}

In [ ]:
skopt = BayesSearchCV(KNeighborsClassifier(), space, cv=5, n_iter=20, random_state=0)

In [ ]:
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)
# %time

skopt.fit(X_train, y_train)

In [ ]:
skopt.score(X_test, y_test)

<font color = "#CC3D3D">
## End